In [ ]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 61.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 75.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 KB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 47.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Prepari

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), 25.78 MiB | 6.36 MiB/s, done.


In [ ]:
import pandas as pd
import numpy as np 
import ktrain
from ktrain import text
import tensorflow as tf


## Loading the dataset

In [ ]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx')
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx')

In [ ]:
data_train.shape, data_test.shape

((25000, 2), (25000, 2))

In [ ]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


## Train_Test_Split & Preprocessing with BERT

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train, 
                   text_column = 'Reviews',
                   label_columns = 'Sentiment',
                   val_df = data_test, 
                   maxlen = 500, 
                   preprocess_mode = 'bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...


/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


###Bert Model Training

In [ ]:
model = text.text_classifier(name = 'bert', 
                             train_data = (X_train, y_train), 
                             preproc = preproc )    #saves the bert model params and features. 

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [ ]:
learner = ktrain.get_learner(model=model, train_data= (X_train, y_train), 
                             val_data = (X_test, y_test), 
                             batch_size = 6)

In [ ]:
#find the best learning rate
# learner.lr_find()
# learner.lr_plot()

# may take days to find out. 

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 4606s 1s/step - loss: 0.2472 - accuracy: 0.8999 - val_loss: 0.1002 - val_accuracy: 0.9684


#### Testing Fine-Tuned Bert Model

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
data = ['this movie was mmmm', 
        'the film sucked. the actors need to pull up their socks', 
        'what a beautiful movie. will definitely be watching it again.']

In [ ]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [ ]:
predictor.predict(data, return_proba=True)

array([[0.6390356 , 0.36096442],
       [0.9960621 , 0.00393785],
       [0.00421936, 0.9957806 ]], dtype=float32)

In [ ]:
predictor.get_classes()

['neg', 'pos']

In [ ]:
predictor.save('/content/bert/')

In [ ]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 11%)
  adding: content/bert/tf_model.preproc (deflated 48%)


In [ ]:
!zip -r "/content/drive/MyDrive/Colab Notebooks/NLP/Model Deployment Course/bert.zip" /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 11%)
  adding: content/bert/tf_model.preproc (deflated 48%)


#### Saving and Loading Fine Tuned Model

In [ ]:
predictor_load = ktrain.load_predictor('/content/bert/')

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
predictor_load.get_classes()

['neg', 'pos']